In [10]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect, brentq
from copy import copy


## Problem 1

$$\begin{aligned}
S_0 &= 90\\
\Delta &= 0.25\\
\Gamma &= 0.015 \\
C &= 2.65\\
\end{aligned}$$
Change in 1% of stock price is 0.9

To rebalance (update) a delta hedge, we need to supply 
$$ d\Delta = \Gamma \times dS = 0.015  \times 0.9 = 0.0135$$

In dollar value,
$$  d\Delta (S+dS) = 0.0135 \times (90+0.9) = 1.23$$


## Problem 2

PDE for $C(K,T)$:
$$\frac{\partial C}{\partial T} = \frac12 K^2 \sigma^2 \frac{\partial^2 C}{\partial K^2} -rK \frac{\partial C}{\partial K}  $$
where $r=0.05, \sigma=0.35$


`2a`



$$\begin{aligned}
&C(15.0, 1.2) = C(15.0, 1.1) + \Delta T \frac{\partial C(15.0, 1.1)}{\partial T} \\
&= C(15.0, 1.1) + \Delta T \left(\frac12 K^2 \sigma^2 \frac{\partial^2 C(15.0, 1.1)}{\partial K^2} -rK \frac{\partial C(15.0, 1.1)}{\partial K} \right)
\\
&\frac{\partial C(15.0, 1.1)}{\partial T} \approx \frac{C(15.0, 1.2) - C(15.0, 1.1)}{\Delta T} \\
&\frac{\partial C(15.0, 1.1)}{\partial K} \approx\frac{C(15.5, 1.1) - C(14.5, 1.1)}{2\Delta K} \\
&\frac{\partial^2 C(15.0, 1.1)}{\partial K^2} \approx\frac{C(15.5, 1.1) - 2C(15.0, 1.1) + C(14.5, 1.1)}{(\Delta K)^2} 
\end{aligned}$$



In [2]:
C_u = 2.33
C_m = 2.55
C_d = 2.79
K_u = 15.5
K_m = 15.0
K_d = 14.5
T_1 = 1.1
T_2 = 1.2
r = 0.05
sigma = 0.35
dcdk = 0.5 * (C_u - C_d) / (K_u - K_d)
dcdkk = (C_u - 2 * C_m + C_d) / (K_u - K_m)**2
a = 0.5 * K_m**2 * sigma**2 * dcdkk
b = r * K_m * dcdk
print(C_m + (T_2 - T_1) * (a - b))

2.677500000000002


`2b`


$$ p(15.0, 1.1) = \frac{\partial^2 C}{\partial S^2} = \frac{\partial^2 C}{\partial K^2} \approx\frac{C(15.5, 1.1) - 2C(15.0, 1.1) + C(14.5, 1.1)}{(\Delta K)^2} $$

In [ ]:
print(dcdkk)

0.08000000000000185


## Problem 3

Let $S$ be a non-dividend-paying stock that follows Geometric Brownian motion with $70\%$ volatility and time-$0$ price $S_{0} = 10$. The interest rate is $2\%$

Find the time-$0$ price of a contract which pays at time $1$

$$ \left(\frac{S_{0.5}+S_{1.0}}{2}- 12 \right)^+$$

Use conditional Monte Carlo, conditioning on $S_{0.5}$. 

Let:
- $T_{0} = 0.5$
- $T_{1} = 1$
- $S_{0} = 10$
- $K = 12$
- $r = 0.02$
- $R_{grow} = r$
- $\sigma = 0.70$

The conditional expected discounted payoff at time 0, given $(S_{T_{0}})$, can be expressed as:
$$\text{EY}\left(S_{T_{0}}\right) = e^{-rT_{0}} \cdot \mathbb{E}\left[Y\left(S_{T_{1}}, S_{T_{0}}\right) | S_{T_{0}}\right] 
= e^{-rT_{0}} \cdot \mathbb{E}\left[\left(\frac{S_{T_0}+S_{T_1}}{2}- 12 \right)^+ | S_{T_{0}}\right] 
$$

Note: We need to consider the additional condition when $\left(S_{T_{1}} > 2K\right)$.

Our final form for the time-$0$ price is the following:
$$\begin{align*}
C^{BS} &= C^{BS}\left(S_{t}=S_{T_{0}}, t=T_{0}, K=K, T=T_{1}, R_{grow}, r, \sigma\right) \\
CB^{BS} &= \left( S_{T_{0}} - K\right) \\
d_{2} &= \frac{\ln\left(\frac{S_{T_{0}}}{2K}\right) + \left(R_{grow} - \frac{1}{2}\sigma^{2}\right)\left(T_{1}-T_{0}\right)}{{\sigma\sqrt{T_{1}-T_{0}}}} \\
V_{0} &= \exp\left(-rT_{0}\right)\left[\mathbb{1}\{S_{T_{0}} \leq 2K \}C^{BS} + \mathbb{1}\{S_{T_{0}} > 2K\}CB^{BS} \right] \\
\end{align*}$$


In [ ]:
class GBM:

    def __init__(self,S0,r,sigma=None):
        self.S0=S0
        self.r=r
        self.sigma=sigma


In [ ]:
class CallOnAverage:

    def __init__(self,K,T0,T1):
        self.K = K
        self.T0 = T0
        self.T1 = T1

In [ ]:
### Pricing Engine
class AnalyticEngine:

    def __init__(self):
        pass
    
    
    def BSpriceCall(self, dynamics: GBM, contract: CallOnAverage) -> float:
        ### Separate dynamics and contract class attributes to variables
        K, T0, T1 = contract.__dict__.values()
        S0, r, sigma = dynamics.__dict__.values()
        
        ### Make BS variables and call price
        timeRemaining = T1 - T0
        F = S0 * np.exp(r * timeRemaining)
        std = sigma * np.sqrt(timeRemaining)
        d1 = np.log(F / K) / std + std / 2
        d2 = d1 - std
        call_price = np.exp(-r * timeRemaining) * (F * norm.cdf(d1) - K * norm.cdf(d2))
        
        return call_price
    

class MCengine:

    def __init__(self,M,seed):
        self.M = M  #number of paths
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible

    def randomLogreturn(self,dynamics,deltat):
        return ((dynamics.r - dynamics.sigma**2/2) * deltat 
                + dynamics.sigma*np.sqrt(deltat) * self.rng.normal(size=self.M))

    def price_CallOnAverage_GBM(self,contract,dynamics):
        #You fill this in.
        K, T0, T1 = contract.__dict__.values()
        S0, r, sigma = dynamics.__dict__.values()
        deltat1 = T0
        deltat2 = T1 - T0
        
        # Simulate S0.5
        S0_5 = S0 * np.exp(self.randomLogreturn(dynamics, deltat1))
        
        payoffs = np.zeros(self.M)
        
        # Simulate S1.0 given S0.5
        for i in range(self.M):
            if S0_5[i] < 24:
                S1_0 = S0_5[i] * np.exp(self.randomLogreturn(dynamics, deltat2)[i])
                payoffs[i] = max((S0_5[i] + S1_0)/2 - K, 0)
            else:
                payoffs[i] = S0_5[i] - 12
        
        # Discount payoffs to present value
        payoffs *= np.exp(-r * T1)
        
        # Calculate price and standard error
        price = np.mean(payoffs)
        standard_error = np.std(payoffs) / np.sqrt(self.M)

        return (price, standard_error)


In [ ]:
p3dynamics = GBM(sigma=0.70,S0=10,r=0.02)

In [ ]:
p3contract = CallOnAverage(K=12,T0=0.5,T1=1.0)

In [ ]:
p3MC = MCengine(M=10000,seed=0)
(p3price, p3standard_error) = p3MC.price_CallOnAverage_GBM(p3contract,p3dynamics)
print(p3price, p3standard_error)

1.5324553557623568 0.037832512015875994


## Problem 4

In this game, 
- if at $n=1$ we roll 6 or 5, exercise immediately. Delaying this has probability of $4/6$
- if at $n=2$ we roll 6 or 5 or 4, exercise immediately. Delaying this has probability of $3/6$
- so probability of exercising at $n=3$ is
$$\frac46 \cdot \frac36 = \frac13$$

n = 4
| R | Exercised Once | Not Exercised |
|---|---------------|----------------|
| 6 |      6        |        6       |
| 5 |      5        |        5       |
| 4 |      4        |        4       |
| 3 |      3        |        3       |
| 2 |      2        |        2       |
| 1 |      1        |        1       |
| E |     3.5       |        3.5     |

n = 3
| R | Exercised Once | Not Exercised |
|---|---------------|----------------|
| 6 |      6        |        6       |
| 5 |      5        |        5       |
| 4 |      4        |        4       |
| 3 |      3.5        |        3       |
| 2 |      3.5        |        2       |
| 1 |      3.5        |        1       |
| E |     4.25       |        3.5     |


n = 2
| R | Exercised Once | Not Exercised |
|---|---------------|----------------|
| 6 |      6        |        6       |
| 5 |      5        |        5       |
| 4 |      4.25        |        4       |
| 3 |      4.25        |        3.5       |
| 2 |      4.25        |        3.5       |
| 1 |      4.25        |        3.5       |
| E |     4.67       |        4.25     |


n = 1
| R | Exercised Once | Not Exercised |
|---|---------------|----------------|
| 6 |      -        |        6       |
| 5 |      -        |        5       |
| 4 |      -        |        4.67       |
| 3 |      -        |        4.67       |
| 2 |      -        |        4.67       |
| 1 |      -        |        4.67       |
| E |     -       |        4.95     |


Expectation is:
$$\begin{aligned} 
\mathbb{E}(n=1) &= \mathbb{E} (6+5+ 4 \times \mathbb{E} (n=2))\\
&=  \mathbb{E} (6+5+ 4 \times \mathbb{E} (6+5+4 \times \mathbb{E} (n=3)))\\
&=  \mathbb{E} (6+5+ 4 \times \mathbb{E} (6+5+4 \times \mathbb{E} (6 + 5 + 4 + 3\times \mathbb{E} (n=4))))\\
\end{aligned}$$

In [7]:
print(np.mean([6, 5, 4.67, 4.67, 4.67, 4.67]))

4.946666666666666


## Problem 5

In [17]:
class GBMdynamics:

    def __init__(self, S, r, rGrow, sigma=None):
        self.S = S
        self.r = r
        self.rGrow = rGrow
        self.sigma = sigma

    def update_sigma(self, sigma):
        self.sigma = sigma
        return self

In [16]:
class CallOption:

    def __init__(self, K, T, price=None):
        self.K = K
        self.T = T
        self.price = price

In [15]:
class AnalyticEngine:

    def __init__(self):
        pass

    def BSpriceCall(self, dynamics, contract):
        # ignores contract.price if given, because this function calculates price based on the dynamics

        F = dynamics.S*np.exp(dynamics.rGrow*contract.T)
        std = dynamics.sigma*np.sqrt(contract.T)
        d1 = np.log(F/contract.K)/std+std/2
        d2 = d1-std
        return np.exp(-dynamics.r*contract.T)*(F*norm.cdf(d1)-contract.K*norm.cdf(d2))

    def IV(self, dynamics, contract):
        # ignores dynamics.sigma, because this function solves for sigma.

        if contract.price is None:
            raise ValueError('Contract price must be given')

        df = np.exp(-dynamics.r*contract.T)  #discount factor
        F = dynamics.S / df
        lowerbound = np.max([0,(F-contract.K)*df])
        C = contract.price
        if C<lowerbound:
            return np.nan
        if C==lowerbound:
            return 0
        if C>=F*df:
            return np.nan

        dytry = copy(dynamics)
        # We "try" values of sigma until we find sigma that generates price C

        # First find lower and upper bounds
        sigma_try = 0.2
        while self.BSpriceCall(dytry.update_sigma(sigma_try),contract)>C:
            sigma_try /= 2
        while self.BSpriceCall(dytry.update_sigma(sigma_try),contract)<C:
            sigma_try *= 2
        hi = sigma_try
        lo = hi/2
        # We have calculated "lo" and "hi" which bound the implied volatility from below and above.
        # In other words, the implied volatility is somewhere in the interval [lo,hi].
        # Then, to calculate the implied volatility within that interval,
        # for purposes of this homework, you may either (A) write your own bisection algorithm,
        # or (B) use scipy.optimize.bisect or (C) use scipy.optimize.brentq
        # You will need to provide lo and hi to those solvers.
        # There are other solvers that do not require you to bound the solution
        # from below and above (for instance, scipy.optimize.fsolve is a useful solver).
        # However, if you are able to bound the solution (of a single-variable problem),
        # then bisection or Brent will be more reliable.

        # Option B: use scipy.optimize.bisect
        # Need to pass the function that takes the implied volatility as a single argument
        # Define a function for the call price as a function of sigma
        def func(sigma):
            return self.BSpriceCall(dytry.update_sigma(sigma), contract) - C

        impliedVolatility = bisect(func, lo, hi)

        return impliedVolatility


$$IV^2(T)= \frac{1}{T} \int_0^T \sigma^2(t) dt$$

In [25]:
T1 = 0.3
T2 = 0.4
T3 = 0.6
S0 = 100
IV3 =  0.3
sigma1 = 0.34
C2 = 8.01

In [34]:
#Test the BSpriceCall function
final_analytic = AnalyticEngine()

# C3 from IV3
dynamics3 = GBMdynamics(sigma=IV3, rGrow=0, S=S0, r=0)
contract3 = CallOption(K=100, T=T3)
callprice3 = final_analytic.BSpriceCall(dynamics3, contract3)
print('C3', callprice3)
C3 = callprice3

# IV2 from C2
contract2 = CallOption(K=100, T=T2)
contract2.price = C2
dynamics2 = GBMdynamics(sigma=IV3, rGrow=0, S=S0, r=0)
IV2 = final_analytic.IV(dynamics2,contract2)    # This code, EXACTLY AS WRITTEN HERE, must execute without crashing
print('IV2', IV2)

# Use the integral to relate sigma and IV
IV1 = sigma1
dynamics1 = GBMdynamics(sigma=IV1, rGrow=0, S=S0, r=0)
contract1 = CallOption(K=100, T=T1)
callprice1 = final_analytic.BSpriceCall(dynamics1, contract1)
print('C1', callprice1)
C1 = callprice1

# calculate sigma2
sigma2 = ((IV2**2 * T2 - sigma1**2 * T1) / (T2 - T1))**0.5
print('sigma2', sigma2)

# calculate sigma3
sigma3 = ((IV3**2 * T3 - sigma1**2 * T1 - sigma2**2 * (T2 - T1)) 
          / (T3 - T2))**0.5
print('sigma3', sigma3)

C3 9.24976421293605
IV2 0.3179976498926408
C1 7.418607894112
sigma2 0.2401874712572872
sigma3 0.26029788574922175


In [36]:
print('IV1, IV2, IV3: ', IV1, IV2, IV3)
print('sigma1, sigma2, sigma3: ', sigma1, sigma2, sigma3)
print('C1, C2, C3: ', C1, C2, C3)

IV1, IV2, IV3:  0.34 0.3179976498926408 0.3
sigma1, sigma2, sigma3:  0.34 0.2401874712572872 0.26029788574922175
C1, C2, C3:  7.418607894112 8.01 9.24976421293605
